# Regression

The goal of this interactive demo is to show you how a machine learning model can perform regression. However, keep in mind, that the code in this notebook was simplified for the demo, and should not be used as a plug and play example for real machine learning projects.

In this notebook we will explore two different types of regression models:

- `Ridge` regression, using an **L2** norm (i.e. euclidean distance) to regularize its coefficients
- `Lasso` regression, using an **L1** norm (i.e. absolute distance) to regularize its coefficients


## The dataset

For the purpose of this demo we will use the **house prices** dataset about residential homes in Ames, Iowa. As before, we will simplify the dataset slightly to keep computation low and interpretation easy.

Let's go ahead and download and prepare the dataset:

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np

# Load house prices dataset
df = pd.read_csv('house_prices.csv')
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

# Remove rows with missing values
X = X.dropna(axis=1)

# One-hot encode categorical and text features
X = pd.get_dummies(X)

# Log-scale target feature, i.e. the price of the house
y = np.log10(y)

# Show 5 random rows in the dataset
X.sample(5)

Let's extract the size of the dataset and plot a value distribution of the target feature.

In [ ]:
# Report size of the dataset
print(f"Dimension of X: {X.shape}\nDimension of y: {y.shape}")

# Plot value distribution of target feature
import matplotlib.pyplot as plt

plt.hist(y, bins=100)
plt.title("House price [log10 scaled]")

Last but not least, let's split the dataset into a training and test set, so that we can better fine tune the hyper-parameters. The split will be 80 / 20, meaning that we use 80% of the dataset as training set and 20% as test set.

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)

## Machine learning model pipeline

First things first, let's specify the type of regression model we want to use (you can choose between `Ridge` and `Lasso`).

In [ ]:
from sklearn.linear_model import Ridge, Lasso
regressor = Ridge()

Now that we know which regression model we will be using, let's also create the processing pipeline that comes before it. Such a pipeline allows us to fine-tune the hyper-parameters of the regression model, as well as explore the usefullness of different pre-processing routines.

For this demo, let's explore different data scaling routines, as well as usage of a principal component analysis (PCA) step.

In [ ]:
# Import relevant packages
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer

# Processing pipeline for numerical data
numeric_preprocessor = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA()),
    ]
)

# Combine this with the processing pipeline for categorical data
preprocessor = ColumnTransformer(
    [
        ("num", numeric_preprocessor, X.select_dtypes("number").columns),
        ("cat", OneHotEncoder(handle_unknown="ignore"),
         X.select_dtypes(exclude="number").columns),
    ],
    remainder="passthrough",
)

# Add regression model at the end of the pipeline
pipe = Pipeline(steps=[("prep", preprocessor), ("regressor", regressor)])

## Train model

The last thing that we need to specify before we can train the model, is the parameter grid that we want to explore. In other words, what kind of hyper-parameter combinations do we want to explore?

In [ ]:
from sklearn.model_selection import ParameterGrid

# Create parameter grid
param_grid = {
    # Explore different types of scalers
    f"prep__num__scaler": [None, StandardScaler()],
    
    # Explore usefullness of a PCA
    f"prep__num__pca": [None, PCA(0.99)],
    
    # Fine-tune regressor hyper-parameter 'alpha'
    "regressor__alpha": np.logspace(-5, 5, 11),
}

print(
    f"The grid search will explore {len(ParameterGrid(param_grid))} paramter combinations."
)

Now, everything is ready to train. So let's go ahead!

In [ ]:
from sklearn.model_selection import GridSearchCV

# Put parameter grid and regression model into GridSearchCV
grid = GridSearchCV(
    pipe, param_grid, cv=2, n_jobs=-1, verbose=2, return_train_score=True
)

# Train regression model on training data
res = grid.fit(X_tr, y_tr)

<div class="alert alert-info">
  <strong>Important note:</strong> Potential <code>RuntimeWarnings</code> can be ignored!
</div>

## Grid search exploration

Once the parameter grid was explored, let's have a look at the parameter combinations that lead to the best models (i.e. the models with the highest performance scores).

In [ ]:
# Clean out results table of the parameter grid
cv_results = pd.DataFrame(res.cv_results_)
cv_results = cv_results.iloc[
    :, ~cv_results.columns.str.contains("time|split[0-9]*|rank|params")
]
new_columns = [c.split("param_")[1] if "param_" in c else c for c in cv_results.columns]
new_columns = [c.split("prep__")[1] if "prep__" in c else c for c in new_columns]
cv_results.columns = new_columns

# Sort results table according to best score
cv_results = cv_results.sort_values("mean_test_score", ascending=False)

# Visualize top parameter combinations and scores
cv_results.head(10)

Let's have a closer look at this table and the different grid-search parameter combinations by plotting them.

In [ ]:
from utils import plot_grid_search
plot_grid_search(param_grid, cv_results)

## Model performance investigation

Depending on what we see in the previous table and figures, we might want to readjust our grid-search and retrain the model. But once we're happy with what we did and think we identified the "best" optimal model, we can go ahead and test it against a new dataset and explore the relevance of the features.

In [ ]:
# Select the best estimator with the best hyper parameter
best_estimator = grid.best_estimator_

# Fit this best estimator once more, but this time using the full training set
_ = best_estimator.fit(X_tr, y_tr)

Now we have the final model, the model that we potentially would want to deploy on our devices. So let's see how well it would perform on the withheld test set.

In [ ]:
# Evaluate model performance on training and test set
score_tr = best_estimator.score(X_tr, y_tr)
score_te = best_estimator.score(X_te, y_te)

print(f"Prediction score on train data: {score_tr:.3f}\n\
Prediction score on test data:  {score_te:.3f}")

The training and the test scores are very close, that is great news! This implies that our model probably is able to generalize it's performance onto a new and unseen dataset.

## Coefficient exploration

A great way to better understand a model's performance is to look at its coefficients. However, it is **important to highlight that the biggest coefficients do not always have to be the most important ones**.

So what are our model's coefficients?

In [ ]:
# Plotting of regression model coefficient
plt.figure(figsize=(15, 25))
sort_idx = np.argsort(best_estimator["regressor"].coef_)
plt.barh(best_estimator.feature_names_in_[sort_idx],
         best_estimator["regressor"].coef_[sort_idx])
plt.ylim(-1, sort_idx.max() + 1)
plt.tight_layout()

Now, to better understand **which coefficients are really important** for the prediction, we can deploy a **permutation approach**. In short, we will run the model prediction multiple times, but for each iteration we shuffle one of the features randomly. The idea is the following, if a feature is important, than the random shuffling of this feature should reduce the overall performance score.

In [ ]:
from sklearn.inspection import permutation_importance

# Reshuffle each feature 5 times and compute the drop of the performance score
result = permutation_importance(
    best_estimator, X_te, y_te, n_repeats=5, random_state=0, n_jobs=-1
)

Once everything is computed, we can go ahead and plot the feature importance for each feature. The further away this feature importance score is from zero, the more important is its role for a good model performance.

In [ ]:
# Plot model performance
plt.figure(figsize=(15, 25))
sorted_idx = result.importances_mean.argsort()
plt.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=X_te.columns[sorted_idx]
)
plt.title("Feature importances score (test set)")
plt.tight_layout()

<div class="alert alert-success">
  <h2>Exercise</h2>
    <p></p>
Change the <code>regressor</code> parameter to <code>Lasso()</code> and rerun all the cells after that. How does this effect the final score? How does this change the importance and size of the model coefficients?
</div>